## Idea

We need a backup that is:
- Fast
- Consistent
- Continuous in time (to "redo" operations and create a "warm" standby)

We do a physical backup which is fast and help it to be consistent using WAL archiving that can bring us from the inconsistent state of the physical backup to any consistent point in time recorded in the WAL archives

## Advantages

- Fast since physical backup is fast
- Easily consistent with the WAL files
- Supports recovering to a specific consistent point in time which can help to do a `REDO` like operation
- Don't have to do large backups frequently

## Disadvantages

- Hard to configure and maintain
- WAL files can be very large so it's storage consuming

## Flow

Notes:
- Since CHECKPOINTS are performed often and a backup is taking time, the particular CHECKPOINT should be stored separately (backup_label file) from pg_control because other CHECKPOINTS can be performed during backup
- Backups can have a label (name)
- Backup history file is replacing backup label because the backup WAL files are already archived after `pg_backup_stop` which calls `pg_switch_wal`.
- Backup history name is determined by the actual WAL file and LSN that should be applied first after restoring the physical backup\
In a format: `%wal_file_name.%lsn.backup` something like: `0000000100001234000055CD.007C9330.backup`

## Maintenance

### Choosing Interval Between Backups

Potentially, WAL files could be stored indefinitely so theoretically you could only one physical backup.

But, you should consider:
- The larger the difference between the current point in time and the physical backup, the larger the amount of WAL records should be applied on recovery
- Larger interval also means a lot of storage used for storing WAL files

You should 

### Multiple Backups

Consider storing more than one working backup for:
- More reliability, if anything is wrong with the last one this can be a life saver (in practice shouldn't really happen)
- Larger points in time that can be recovered, the only recoverable points of time are the ones that can be reached are after the earliest physical backup

### Backup Configuration Files

WAL files don't include configuration files (`postgresql.conf`, `pg_hba.conf` and `pg_ident.conf`) changes / content since those are not edited with SQL queries

Make sure to backup them also in a suitable place!